In [17]:
import pyvista as pv
import numpy as np
import glob
import vtk
import os
import requests

## Auxiliary functions

In [31]:
ZENODO_BASE_URL = "https://zenodo.org/record/16995252/files/" 
SUBEND_VTU_FILE = "sUbend_modified.vtu"

In [34]:
def download_file(file_name, folder_path='.'):
    url = ZENODO_BASE_URL + file_name
    local_path = os.path.join(folder_path, file_name)
    
    r = requests.get(url, stream=True)
    if r.status_code != 200:
        print(f"ERROR: could not download {file_name} (status {r.status_code})")
        return False
    
    with open(local_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    print(f"{file_name} downloaded successfully.")
    return True

def create_VTKlegacy_files(folder_path, extension='.vtu'):
    """
    Check if SUBEND_VTU_FILE exists in folder_path (should be 'VTK/').
    If not, download it from Zenodo to folder_path.
    Then convert to VTK legacy format and save the .vtk file one level above folder_path.
    """
    # The VTU file should be in folder_path (e.g., './VTK')
    vtu_file_path = os.path.join(folder_path, SUBEND_VTU_FILE)
    if not os.path.exists(vtu_file_path):
        print(f"{vtu_file_path} not found locally. Attempting to download from Zenodo...")
        success = download_file(SUBEND_VTU_FILE, folder_path)
        if not success:
            print(f"Skipping {vtu_file_path} (could not download).")
            return
        if not os.path.exists(vtu_file_path):
            print(f"Downloaded file {vtu_file_path} still not found. Skipping.")
            return

    try:
        mesh = pv.read(vtu_file_path)
    except Exception as e:
        print(f"Failed to read {vtu_file_path}: {e}")
        return

    base_name = os.path.splitext(os.path.basename(vtu_file_path))[0]
    # Save the .vtk file one level above folder_path
    parent_dir = os.path.abspath(os.path.join(folder_path, os.pardir))
    vtk_file_name = os.path.join(parent_dir, base_name + '.vtk')

    # Save the VTK legacy file
    writer = vtk.vtkDataSetWriter()
    writer.SetFileTypeToASCII()
    writer.SetInputData(mesh)
    writer.SetFileName(vtk_file_name)
    writer.Write()
    print(vtk_file_name)

def create_seed_file(points, seedfile):
  with open(seedfile, 'w') as file:
    for i, point in enumerate(points):
      file.write(f'{point[0]} {point[1]} {point[2]}\n')
      
def sample_points_inside_mesh(mesh, n_points, threshold=1e-3):
    xmin, xmax, ymin, ymax, zmin, zmax = mesh.bounds
    points_inside = []
    # Keep sampling until you have enough points inside
    while len(points_inside) < n_points:
        n_sample = max(n_points - len(points_inside), n_points // 2)
        points = np.column_stack([
            np.random.uniform(xmin, xmax, n_sample),
            np.random.uniform(ymin, ymax, n_sample),
            np.random.uniform(zmin, zmax, n_sample)
        ])
        enclosed = pv.PolyData(points).select_enclosed_points(mesh, tolerance=0.0)
        mask = enclosed['SelectedPoints'] == 1
        filtered_points = enclosed.points[mask]
        distances = pv.PolyData(filtered_points).compute_implicit_distance(mesh)
        mask = np.abs(distances['implicit_distance']) > threshold
        selected_points = filtered_points[mask]
        points_inside.extend(selected_points.tolist())
        print(f'Sampled {len(points_inside)} / {n_points} points inside the mesh...')
    return np.array(points_inside[:n_points]), distances

## Prepare geometry to work with VTK-m

In [36]:
# The function create_VTKlegacy_files converts all VTU files in a given folder to VTK legacy format.
# It searches for files with the '.vtu' extension in the specified folder and its subfolders.
# For each VTU file found, it reads the mesh, writes it as a VTK legacy file (ASCII format), and saves a list of files in txt format.
# The output file is named 'flowmap_<suffix>.vtk', where <suffix> is derived from the original filename.
# It also records the mapping between the new VTK filename and the mesh's 'TimeValue' attribute in a text file (list_of_files.txt).
# The function returns 0 after processing all files.
create_VTKlegacy_files('./VTK', extension='.vtu')

./VTK/sUbend_modified.vtu not found locally. Attempting to download from Zenodo...
sUbend_modified.vtu downloaded successfully.
/home/pablov/spinAdvection/examples/sUbend_mwe/sUbend_modified.vtk


It is possible to work with both .vtk and image files (.vts). To convert vtk to image files,  

In [21]:
# Define the folder path
folder_path = "./"
destination_path = './'

# Get a list of all VTK files in the folder
_files = [file for file in os.listdir(folder_path) if file.endswith(".vtk")]

# Loop over each VTK file
for i, _file in enumerate(_files):
  # Load the file
  mesh = pv.read(os.path.join(folder_path, _file))

  Coords_0  = mesh.points
  scaling=1
  image_box = [scaling*(np.max(Coords_0[:,0])-np.min(Coords_0[:,0])),\
          scaling*(np.max(Coords_0[:,1])-np.min(Coords_0[:,1])),\
          scaling*(np.max(Coords_0[:,2])-np.min(Coords_0[:,2]))]

  offset = 0
  orig_image =[np.min(Coords_0,axis=0)[0]+offset,\
          np.min(Coords_0,axis=0)[1]+offset,\
          np.min(Coords_0,axis=0)[2]+offset]

  edge_size = .5e-3
  image_sizex=edge_size
  image_sizey=edge_size
  image_sizez=edge_size

  nx = int(image_box[0]/image_sizex)
  ny = int(image_box[1]/image_sizey)
  nz = int(image_box[2]/image_sizez)

  res_x = image_box[0]/nx
  res_y = image_box[1]/ny
  res_z = image_box[2]/nz

  dims=(nx, ny, nz)
  spa=(res_x,res_y,res_z)
  orig=(orig_image[0],orig_image[1],orig_image[2])
  image = pv.ImageData(dimensions=dims,spacing=spa,origin=orig)

  mesh=mesh.cell_data_to_point_data()
  probed_image=image.sample(mesh, mark_blank=False)
  array_data=probed_image.point_data
  #probed_image.clear_arrays()
  probed_image['U']=array_data['U']
  probed_image=probed_image.point_data_to_cell_data()
  mesh = probed_image

  # Save the VTK legacy file
  writer = vtk.vtkDataSetWriter()
  writer.SetFileTypeToASCII()
  writer.SetInputData(mesh)
  # Set the output file name with .vts extension
  output_file = _file.rsplit('.vtk')[0]+'.vts'
  writer.SetFileName(folder_path + '/' + output_file)
  print(i, _file,  output_file)
  writer.SetFileVersion(42)
  writer.SetWriteArrayMetaData(0)
  writer.Write()

Now it is possible to run the computation using this image files instead of VTK files (faster, but probably less accurate). To do so, it is neccesary to change the file in advection_settings.json

## Generate random internal seeds

In [22]:
fov_mesh = pv.read('./fov.vtu')
surface = fov_mesh.extract_surface()

In [23]:
surface

PolyData (0x7fd496d6f3a0)
  N Cells:    49080
  N Points:   49082
  N Strips:   0
  X Bounds:   2.183e-03, 1.414e-01
  Y Bounds:   -1.031e-01, 1.603e-02
  Z Bounds:   -1.603e-02, 1.602e-02
  N Arrays:   2

In [24]:
# Create random number of seeds within the peeled surface
num_seeds = 10000

points, _ = sample_points_inside_mesh(surface, num_seeds, threshold=0.5e-3)


Sampled 3795 / 10000 points inside the mesh...
Sampled 6201 / 10000 points inside the mesh...
Sampled 8120 / 10000 points inside the mesh...
Sampled 10027 / 10000 points inside the mesh...


In [25]:
pv.PolyData(points).save('internal_seeds.vtp')

In [26]:
create_seed_file(points, 'internal_seeds.txt')

## Generate seeds for reseeding at entrance

In [27]:
inflow_mesh = pv.read('./inflow.vtu')
inflow_surface = inflow_mesh.extract_surface()

In [28]:
num_seeds = 1000

points, _ = sample_points_inside_mesh(inflow_surface, num_seeds, threshold=0.005e-3)


Sampled 786 / 1000 points inside the mesh...
Sampled 1191 / 1000 points inside the mesh...


In [29]:
pv.PolyData(points).save('inflow_seeds.vtp')

In [30]:
create_seed_file(points, 'entrance_seeds.txt')